In [8]:
import sys
import os
import numpy as np
import matplotlib.pylab as plt 
import scipy.signal as sg
import scipy.stats as stats
import pyodbc
import xlsxwriter
import pandas as pd
import datetime
import csv

In [ ]:
"""PyQt5 notes for GUI
All functions will be connected to different button/options
All if-else functions will be shown as options of bottons or open windows in the future """

In [7]:
def GetInfo(fmt = 'Excel'):
    """This is to export FileMaker Data to Excel spreadsheet. 
    If no argument is given, the function will generate normal spreadsheet for R/Matlab.
    Otherwise, the input should be 'csv' as for csv of Python auto analysis
    This funtion will generate files for further use and will return list of Calibration Condition, Experimental Condition,
    Experimental Treatment, The total number of control and experimental animals
    Currently I did not count control number by Genotype. These numbers will just give a overview. 
    """
    
    #The following is to link with FileMaker database. 
    #This function only works under BCM network or via VPN.
    #The computer used should be set up properly for FileMaker ODBC
    dsn = 'DRIVER=/Library/ODBC/FileMaker ODBC.bundle/Contents/MacOS/fmodbc.so;Server=128.249.80.130;Port=2399;Database=MICE;UID=Admin;PWD='
    mousedb = pyodbc.connect(dsn)
    mousedb.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    mousedb.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    mousedb.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    mousedb.setencoding(encoding='utf-8')
    
    path = str(input("The path of file:"))    #This should be the path where the raw data was saved 
    newpath = path+'/MUID/'    # Individual spreadsheets will be saved under save path MUID subfolder
    try: os.makedirs(newpath) #only generate this subfolder when it is not exist
    except: pass
    os.chdir(newpath)
    #Res = input("Start a new project? (y/n): ") 
    #this is reserved fort future directly input information from stand alone app and crosstalk with FileMaker to generation information
    #if Res == "n":

    ProName = input("Project name: ") #This name is supposed to be exactly same as the one in FileMaker under Project Name
    #The following is call information from FileMaker using ODBC and SQL querry 
    cursor = mousedb.execute('select "MUID", "Sex","Genotype","Group","Weight","Age","DOB",\
                             "Start_body_temperature","Mid_body_temperature","End_body_temperature","post30_body_temperature",\
                             "Room_Temp","Rotometer_Flowrate","Calibration_Volume","Bar_Pres","Experimental_Date", \
                             "Calibration_Condition","Experimental_Condition","Experiment_Treatment", "Habituation" \
                             from "Plethysmography" where "Experiment_Name" =?', ProName)
    mouse_list = cursor.fetchall()
    Con = 0
    Exp = 0
    GenGroup =[]
    MUIDs = [] #set up MUIDs list to use in other functions

    for row in mouse_list:
        MUIDs.append(row.MUID) #Generate list of MUIDs 
        if row.Group == "Exp": 
            Exp = Exp + 1
            if Exp ==1:
                GenGroup.append(row.Genotype)
            else: pass
        elif row.Group == "Con": 
            Con = Con +1
            if row.Genotype not in GenGroup:
                GenGroup.append(row.Genotype)
            else: pass
        #This is set up for experiments without Mid body temperature
        try: TempAverage1 = row.Start_body_temperature+row.Mid_body_temperature
        except: TempAverage1 = row.Start_body_temperature+row.End_body_temperature
        try: TempAverage2 = row.End_body_temperature+row.Mid_body_temperature
        except: TempAverage2 = TempAverage1
        TempRoom = (row.Room_Temp-32)*5/9
        BarometricPressure = row.Bar_Pres*25.4
        MouseWVPressure1 = 1.142 + (0.8017*TempAverage1) - (0.012*TempAverage1**2) + (0.0006468*TempAverage1**3)
        MouseWVPressure2 = 1.142 + (0.8017*TempAverage2) - (0.012*TempAverage2**2) + (0.0006468*TempAverage2**3)
        try: MouseWVPressureCNORA = 1.142 + (0.8017*TempMiddle) - (0.012*TempMiddle**2) + (0.0006468*TempMiddle**3);
        except: MouseWVPressureCNORA = 'NA' 
        if row.Rotometer_Flowrate == 95: FlowRate = 0.19811
        else: FlowRate == float(input("Flow Rate (SLPH): "))
        if(fmt == 'Excel'): 
            #The following write rows are for current spreadsheet used by MATLAB and R 
            filename = row.MUID+".xlsx"
            workbook = xlsxwriter.Workbook(filename)
            worksheet1 = workbook.add_worksheet()
            content1 = (["Sel Start","Sel End","Sel Duration","Avg Freq","Avg Period","Avg Height","O2 Mean","CO2 Mean","Temp Mean","Cmt Text"])
            worksheet2 = workbook.add_worksheet()
            content2 = (["Parameters","Values"],
                       ["Mouse ID",row.MUID],
                       ["Sex",row.Sex[0].lower()],
                       ["Genotype",row.Genotype],
                       ["Group",row.Group],
                       ["Weight (gm)",float(row.Weight)],
                       ["Tbody Start",float(row.Start_body_temperature)],
                       ["Tbody Intermediate",float(row.Mid_body_temperature)],
                       ["Tbody End",float(row.End_body_temperature)],
                       ["T30",float(row.post30_body_temperature)],
                       ["TRoom",float(TempRoom)],
                       ["Barometric Pressure (mmHg)",float(BarometricPressure)],
                       ['Flow Rate (SLPM)',float(FlowRate)],
                       ["Calibration Volume (ml)",float(row.Calibration_Volume/1000)],
                       ["Date of Experiment",str(row.Experimental_Date)],
                       ["Birthdate of animal",str(row.DOB)],
                       ["Age of animal",int(row.Age)])
            row = 0
            col = 0 
            for para, val in (content1):
                worksheet1.write(row, col,     para)
                worksheet1.write(row, col + 1, val)
                row += 1
            row = 0
            col = 0     
            for para, val in (content2):
                worksheet2.write(row, col,     para)
                worksheet2.write(row, col + 1, val)
                row += 1
            workbook.close()
            #The following writerows are for future use to match with plethysmography analysis by python
        elif(fmt == 'csv'):    
            #writing information to MUID specific data file for future use
            filename_new = row.MUID+".csv"
            file_new =open(filename_new,'w')
            writer = csv.writer(file_new, lineterminator='\n')
            writer.writerows([["MUID",row.MUID],
                         ["Sex",row.Sex[0].lower()],
                         ["Genotype",row.Genotype],
                         ["Group",row.Group],
                         ["Body Weight (g)",str(row.Weight)],
                         ["Beginning Temperature",str(row.Start_body_temperature)],
                         ["Middle Temperature",str(row.Mid_body_temperature)],
                         ["End Temperature",str(row.End_body_temperature)],
                         ["Post 30min Temperature",str(row.post30_body_temperature)],
                         ["TempAverage1 (C)",str(TempAverage1)],
                         ["TempAverage2 (C)",str(TempAverage2)],
                         ["MouseWVPressureCNORA (mmHg)",str(MouseWVPressureCNORA)],
                         ["MouseWVPressure1 (mmHg)",str(MouseWVPressure1)],
                         ["MouseWVPressure2 (mmHg)",str(MouseWVPressure2)],
                         ["TempRoom (C)",str(TempRoom)],
                         ["BarometricPressure (mmHg)",str(BarometricPressure)],
                         ["Age (days)",str(row.Age)],
                         ["Cal Volume",str(row.Calibration_Volume/1000)],
                         ['Flow Rate',str(FlowRate)]])
            file_new.close()
        
        CalCon = mouse_list[0].Calibration_Condition.split('\r')
        ExpCon = mouse_list[0].Experimental_Condition.split('\r')
        Treatment = mouse_list[0].Experiment_Treatment
        return MUIDs, CalCon, ExpCon, Treatment, GenGroup, Con, Exp

In [40]:
def waveform_analysis(resp):
    """This function aims to remove motion effects (by only analysis stable breaths) and give out informations.
    The input(s) are all waveform numpy arrays that are preprocessed by DataColl function.
    Variable resp should be a 1d array with sampling rates of 1000"""
    #The following two statements is for testing by directly loading txt file of resp waveform      
    #resp = np.genfromtxt('M27699.txt')
    #resp = np.transpose(resp)[1]
    def motiondetect(resp):
        """This subfuction using convolve function to detect baseline shifting caused by motion. 
        The algorithm is modified from stackoverflow and should be studied deeper to fully understand."""
        dary= np.array([*map(float, resp)])
        dary -=np.average(test)
        step = np.hstack((np.ones(len(dary)),-1*np.ones(len(dary))))
        dary_step = np.convolve(dary,step,mode='valid')
        factor = np.ones(len(dary_step)-1) #factor is a 1d array that will be applied to remove motion effects
        for i in range(len(dary_step)-1):
            if  np.abs(dary_step[i]/10000)>0.01: # 0.01 is just a temperoral value that was tested to give best results
                # This value should be changed after deeper understanding of this algorithm
                factor[i] = np.nan
            else: pass
        return factor

    factor = motiondetect(resp)
    exp_start = sg.find_peaks(resp,prominence=0.1) 
    #by using scipy.signal.find_peaks function, we are able to identify all peaks with index information
    #The prominence value of 0.1 is preset and tested to give good results
    #Optimization will be required 
    exp_start = exp_start[0]
    exp_start_nomotion = factor[exp_start]*exp_start
    exp_start_nomotion = exp_start_nomotion[~np.isnan(exp_start_nomotion)].astype(dtype=int)
    #Variable exp_start_nomotaion is all the peaks information after removal of motion effects
    resp_peaks = resp[exp_start_nomotion] #Values of each peaks 
    resp_rev = -resp #reverse respiration information to find reversed peaks/start of each breathing cycle
    resp_low_peaks = sg.find_peaks(resp_rev,prominence=0.005) 
    resp_low_peaks = resp_low_peaks[0] #All reversed peaks. Optimization is required     
    resp_start=np.empty(len(exp_start_nomotion),dtype=int) 
    # initization of variable resp_start, which represents the index information of start of each breathing cycle
    for i in range(len(exp_start_nomotion)):
        #this for loop gather variable resp_start and the start of next breathing cycle (variable exp_next)
        resp_start[i] = resp_low_peaks[np.where(resp_low_peaks<exp_start_nomotion[i])][-1]
        try: 
            exp_next[i] = exp_start[np.where(np.isin(exp_start,exp_start_nomotion[i])+1)]
        except: exp_next[i] = np.nan
    resp_end=np.empty(len(exp_next),dtype=int)
    # initization of variable resp_end, which represents the index information of end of each breathing cycle (start of the next one)
    for k in range(len(exp_next)):
        resp_end[k] = resp_low_peaks[np.where(resp_low_peaks<exp_next[k])][-1]
    ins_start = resp[resp_start] #values of start of each breathing cycle 
    exp_end = resp[resp_end] #values of end of each breathing cycle 
    cycle_heights_ind = np.abs(resp_peaks - ins_start) #individuel cycle heights 
    cycle_length_ind = np.abs(exp_end - ins_start) #individuel cycle length
    total_times = np.sum(cycle_length_ind)/1000
    frequency = len(resp_peaks)/total_times
    #How to calculate IBI?
    #How to calculate slopes
    
    #ploting to monitor the motion removal
    #for PyQt5, each data file from LabChart should have options to show all analysis results (individual heights, lengths, frequency, IBI, slopes and monitoring plots of peak picking)
    fig,ax = plt.subplots(figsize = [30,8])
    ax.plot(resp)
    ax.plot(exp_start_nomotion,resp_peaks,'x')
    ax.plot(resp_start,start,'ro')
    ax.plot(resp_end,end,'go')
    ylim = plt.ylim(-2.5,2.5)
    
    return cycle_heights_ind, cycle_length_ind, frequency#,IBI, slopes

In [41]:
#redefine datacoll and plot function
#acclimation should remove first 20 min? 
#data should include post-hypoxia/hypercapnia recovery, and sectioned hypoxic response (3 sets, 5 min each set, not including the first 5 min)